### Importación de las librerias necesarias

In [1]:
from ultralytics import YOLO
import cv2

import numpy as np
import os
import glob
import torch
from ultralytics.utils.plotting import Annotator

# OCR Tools
from easyocr import Reader
import pytesseract
import keras_ocr

#from paddleocr import PaddleOCR,draw_ocr

import re

## Tratamiento de imagenes
import cv2

from matplotlib import pyplot as plt


In [2]:
resultados_esperados = []

# ROI Original
resultados_easyocr_og = []
resultados_pytesseract_og = []
resultados_kerasocr_og = []

# ROI Perspective Corrected
resultados_easyocr_og_perspective = []
resultados_pytesseract_og_perspective = []
resultados_kerasocr_og_perspective = []

# ROI Filters
resultados_easyocr_filters = []
resultados_pytesseract_filters = []
resultados_kerasocr_filters = []

# ROI Character Segmentation
resultados_easyocr_cs = []
resultados_pytesseract_cs = []
resultados_kerasocr_cs = []

### Inicialización de los modelos

In [3]:
coco_model = YOLO('../yolov8n.pt')
license_plate_detector = YOLO('../models/license_plate_detector.pt')
ocr_model = YOLO('../models/ocr.pt')

In [4]:
#SELF MADE DATASET
#path = "../datasets/validation_labeled"

# BAZA SLIKA DATASET
path = "../datasets/baza_slika/040603"
#path = "../datasets/baza_slika/070603"
#path = "../datasets/baza_slika/141002"
#path = "../datasets/baza_slika/170902"
#path = "../datasets/baza_slika/180902"
#path = "../datasets/baza_slika/210503"
#path = "../datasets/baza_slika/280503"

# RODOSOL DATASET
#path = "../datasets/tbFcZE-RodoSol-ALPR/RodoSol-ALPR/images/cars-br"

# AOLP
# Subset AC
#path = "../datasets/AOLP/AOLP/Subset_AC/Subset_AC/Subset_AC/Image"
#path = "../datasets/AOLP/AOLP/Subset_LE/Subset_LE/Subset_LE/Image"
#path = "../datasets/AOLP/AOLP/Subset_RP/Subset_RP/Subset_RP/Image"


contenido = os.listdir(path)

imagenes = []
ficheros_json = []


for fichero in contenido:
    if os.path.isfile(os.path.join(path, fichero)) and (fichero.endswith('.jpg') or fichero.endswith('.JPEG')):
        imagenes.append(path+'/'+fichero)
    
    if os.path.isfile(os.path.join(path, fichero)) and (fichero.endswith('.json')):
        ficheros_json.append(path+'/'+fichero)

total_images = len(imagenes)

In [5]:
confianza_umbral = 0.5
coordenadas_cajas = []

#print(predictions)
vehicles = [2,3,5,7]

threshold = 70

## Declaracion inicial easyocr
reader = Reader(lang_list=["es"], gpu=True, verbose=False)

## pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Inicializar el pipeline de keras-ocr
pipeline = keras_ocr.pipeline.Pipeline()

Looking for C:\Users\José Manuel\.keras-ocr\craft_mlt_25k.h5

Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.BILINEAR...)` instead.

Looking for C:\Users\José Manuel\.keras-ocr\crnn_kurapan.h5


In [6]:
def apply_otsu_binarization(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresholded = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresholded

def remove_noise(image):
    denoised = cv2.GaussianBlur(image, (5, 5), 0)
    return denoised

def apply_morphology(image):
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(image, kernel, iterations=1)
    eroded = cv2.erode(dilated, kernel, iterations=1)
    return eroded

def detect_horizontal_lines(image, original_image):

    result_image = original_image.copy()

    # Convertir la imagen a escala de grises si es necesario
    if len(image.shape) > 2 and image.shape[2] > 1:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image  # La imagen ya es una escala de grises
    
    # Aplicar suavizado para reducir el ruido
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Detectar bordes usando Canny
    edges = cv2.Canny(blurred, 50, 150, apertureSize=3)
    
    # Aplicar transformada de Hough para detectar líneas
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 60, 50, 10, maxLineGap=5)

    horizontal_lines = []

        # Filtrar líneas horizontales basadas en la pendiente
    if lines is None:
        print("No se detectaron líneas horizontales.")
        return [], None  # Devolver una lista vacía si no se detectaron líneas
    
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if abs(y2 - y1) < 60 and abs(x2-x1) > 10:  # Considerar líneas casi horizontales (umbral ajustable)
            horizontal_lines.append(line)
            cv2.line(result_image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Dibujar la línea sobre la imagen original

    
    return horizontal_lines, result_image

def calculate_angle_of_inclination(horizontal_lines):
    # Calcular el ángulo de inclinación promedio de las líneas horizontales
    angles = []
    for line in horizontal_lines:
        x1, y1, x2, y2 = line[0]

        # (y2-y1)/(x2-x1) es la forma de calcular la pendiente de la recta,
        # una vez calculada calculamos la arcotangente para obtener el angulo de inclinacion
        angle_rad = np.arctan2(y2 - y1, x2 - x1)
        angle_deg = np.degrees(angle_rad)
        angles.append(angle_deg)
    
    # Calcular el ángulo promedio
    if angles:
        average_angle = np.mean(angles)
    else:
        average_angle = 0.0
    
    return average_angle

def correct_perspective(image, angle_of_inclination):
    # Obtener dimensiones de la imagen
    h, w = image.shape[:2]
    
    # Calcular la matriz de transformación de perspectiva para enderezar la imagen
    rotation_matrix = cv2.getRotationMatrix2D((w // 2, h // 2), -angle_of_inclination, 1.0)
    corrected_image = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
    
    return corrected_image

In [7]:
def apply_correct_perspective(license_plate_ROI):
    
    thresholded_image = apply_otsu_binarization(license_plate_ROI)

    gray=cv2.bilateralFilter(thresholded_image,11,17,17)

    denoised_image = remove_noise(thresholded_image)

    processed_image = apply_morphology(denoised_image)

    # Calcula la mediana de la intensidad de los gradientes
    median_intensity = np.median(processed_image)
        
    # Define los umbrales bajo y alto adaptativos basados en la mediana
    lower_threshold = float(max(0, (1.0 - 0.33) * median_intensity))
    upper_threshold = float(min(255, (1.0 + 0.33) * median_intensity))
        
    # Aplica el detector de bordes Canny con los umbrales adaptativos
    edges = cv2.Canny(thresholded_image, lower_threshold, upper_threshold) 

    horizontal_lines, imagen_lineas_h = detect_horizontal_lines(edges, license_plate_ROI)

    if np.size(horizontal_lines) != 0:

        angle_of_inclination = calculate_angle_of_inclination(horizontal_lines)

        # Corregir la perspectiva de la placa de matrícula basada en el ángulo de inclinación
        corrected_plate_image = correct_perspective(license_plate_ROI, -angle_of_inclination)
    else:
        corrected_plate_image = license_plate_ROI

    return corrected_plate_image



In [8]:
def calculate_bounds(roi):
    x, y, width, height = cv2.boundingRect(roi)
    x_min = x
    y_min = y
    x_max = x + width
    y_max = y + height
    return (x_min, y_min, x_max, y_max)


def is_roi_contained(roi_inner, roi_outer):
    x_min_inner, y_min_inner, x_max_inner, y_max_inner = calculate_bounds(roi_inner)
    x_min_outer, y_min_outer, x_max_outer, y_max_outer = calculate_bounds(roi_outer)

    # Comprobar si el ROI interno está completamente dentro del ROI externo
    return (x_min_inner >= x_min_outer and
            y_min_inner >= y_min_outer and
            x_max_inner <= x_max_outer and
            y_max_inner <= y_max_outer)

def gray_to_color(image):
    return cv2.cvtColor(image, cv2.COLOR_GRAY2BGR) if len(image.shape) == 2 else image


def add_border(image, border_size=10, white=True):
    # Verificar si la imagen es en escala de grises
    if len(image.shape) == 2:
        # Convertir la imagen en escala de grises a una imagen en color (escala de grises replicada en cada canal)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    
    if white:
        # Agregar un borde blanco a la imagen en color
        bordered = cv2.copyMakeBorder(image, border_size, border_size, border_size, border_size, 
                                    cv2.BORDER_CONSTANT, value=(255, 255, 255))  # White border
    else:
        bordered = cv2.copyMakeBorder(image, border_size, border_size, border_size, border_size, 
                                    cv2.BORDER_CONSTANT, value=(0, 0, 0))  # Black border
    
    target_height = image.shape[0]
    target_width = image.shape[1]
    resized_image = cv2.resize(bordered, (target_width, target_height))
    
    return resized_image

In [9]:
def obtain_license_plate_characters(license_plate_ROI):
    gray=cv2.cvtColor(license_plate_ROI,cv2.COLOR_BGR2GRAY)

    resize_image = cv2.resize(gray, None, fx = 3, fy= 3, interpolation = cv2.INTER_CUBIC)

    #blured_image = cv2.GaussianBlur(resize_image, (5,5), 0)

    ret, thresh = cv2.threshold(resize_image, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)

    rect_kern = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))

    dilation = cv2.dilate(thresh, rect_kern, iterations = 1)

    license_plate_ROI_with_filters = gray_to_color(dilation).copy()

    cnts, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    cnts = sorted(cnts,key=cv2.contourArea,reverse=True)

    image2=resize_image.copy()
    
    NumberPlateCount = None

    sorted_contours = sorted(cnts, key = lambda ctr: cv2.boundingRect(ctr)[0])

    list_of_rois = []

    x_anterior, y_anterior, w_anterior, h_anterior = 0,0,0,0

    license_plate_contours = []

    for cnt in sorted_contours:

        roi = None

        x,y,w,h = cv2.boundingRect(cnt)

        height, width = image2.shape

        if height / float(h) > 6: continue

        ratio = h/float(w)

        #if ratio < 1: continue
        if w > 300: continue

        #if w < 20 or h < 20: continue

        area = h * w

        #if area < 5000: continue
        roi = thresh[y-1: y+h, x:x+w]

        roi = cv2.bitwise_not(roi)

        if roi is None:
            continue

        license_plate_contours.append(cnt)

        
        list_of_rois.append(gray_to_color(add_border(roi, white=False)))

    return license_plate_ROI_with_filters, list_of_rois

In [10]:
def cleanup_text_easyocr(text):
	# strip out non-ASCII text so we can draw the text on the image
	# using OpenCV
	return "".join([c if ord(c) < 128 else "" for c in text]).strip()

def clean_text_keras(text):
    # Eliminar espacios extra
    text = ' '.join(text.split())
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales (dejar solo alfanuméricos y espacios)
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)

    return text

In [11]:
resultados_easyocr_og = []
resultados_pytesseract_og = []
resultados_kerasocr_og = []

# ROI Perspective Corrected
resultados_easyocr_og_perspective = []
resultados_pytesseract_og_perspective = []
resultados_kerasocr_og_perspective = []

# ROI Filters
resultados_easyocr_filters = []
resultados_pytesseract_filters = []
resultados_kerasocr_filters = []


def ocr_analysis(ROI_original, ROI_original_perspective_corrected, ROI_with_filters, ROI_character_segmentation):
    
    ##############################
    # OCR ROI Original #
    ##############################

    # easyOCR
    clean_text = ''

    ocr_results = reader.readtext(ROI_original)

    for (bbox, text, prob) in ocr_results:
        # display the OCR'd text and associated probability
        #print("[INFO] {:.4f}: {}".format(prob, text))

        # cleanup the text and draw the box surrounding the text along
        # with the OCR'd text itself
        clean_text = cleanup_text_easyocr(text)
    
    resultados_easyocr_og.append(clean_text)

    # pytesseract
    text_tess = ''

    text_tess = pytesseract.image_to_string(ROI_original, lang='eng', config='-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ —psm 8 —oem 3')

    text_tess = ''.join(e for e in text_tess if e.isalnum()) 

    resultados_pytesseract_og.append(text_tess)

    # keras OCR
    concatenated_text = ''

    #pipeline = keras_ocr.pipeline.Pipeline()

    image = keras_ocr.tools.read(ROI_original)

    predictions = pipeline.recognize([image])[0]

    texts = [prediction[0] for prediction in predictions]

    # Limpiar los textos
    cleaned_texts = [clean_text_keras(text) for text in texts]

    concatenated_text = ''.join(cleaned_texts)

    resultados_kerasocr_og.append(concatenated_text)

    ##############################
    # OCR Perspective Correction #
    ##############################
    # easyOCR
    clean_text = ''

    ocr_results = reader.readtext(ROI_original_perspective_corrected)

    for (bbox, text, prob) in ocr_results:
        # display the OCR'd text and associated probability
        #print("[INFO] {:.4f}: {}".format(prob, text))

        # cleanup the text and draw the box surrounding the text along
        # with the OCR'd text itself
        clean_text = cleanup_text_easyocr(text)
    
    resultados_easyocr_og_perspective.append(clean_text)

    # pytesseract
    text_tess = ''

    text_tess = pytesseract.image_to_string(ROI_original_perspective_corrected, lang='eng', config='-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ —psm 8 —oem 3')

    text_tess = ''.join(e for e in text_tess if e.isalnum()) 

    resultados_pytesseract_og_perspective.append(text_tess)

    # keras OCR
    concatenated_text = ''

    #pipeline = keras_ocr.pipeline.Pipeline()

    image = keras_ocr.tools.read(ROI_original_perspective_corrected)

    predictions = pipeline.recognize([image])[0]

    texts = [prediction[0] for prediction in predictions]

    # Limpiar los textos
    cleaned_texts = [clean_text_keras(text) for text in texts]

    concatenated_text = ''.join(cleaned_texts)

    resultados_kerasocr_og_perspective.append(concatenated_text)


    ##############################
    # OCR Filters #
    ##############################
    # easyOCR
    clean_text = ''

    ocr_results = reader.readtext(ROI_with_filters)

    for (bbox, text, prob) in ocr_results:
        # display the OCR'd text and associated probability
        #print("[INFO] {:.4f}: {}".format(prob, text))

        # cleanup the text and draw the box surrounding the text along
        # with the OCR'd text itself
        clean_text = cleanup_text_easyocr(text)
    
    resultados_easyocr_filters.append(clean_text)

    # pytesseract
    text_tess = ''

    text_tess = pytesseract.image_to_string(ROI_with_filters, lang='eng', config='-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ —psm 8 —oem 3')

    text_tess = ''.join(e for e in text_tess if e.isalnum()) 

    resultados_pytesseract_filters.append(text_tess)

    # keras OCR
    concatenated_text = ''

    #pipeline = keras_ocr.pipeline.Pipeline()

    image = keras_ocr.tools.read(ROI_with_filters)

    predictions = pipeline.recognize([image])[0]

    texts = [prediction[0] for prediction in predictions]

    # Limpiar los textos
    cleaned_texts = [clean_text_keras(text) for text in texts]

    concatenated_text = ''.join(cleaned_texts)

    resultados_kerasocr_filters.append(concatenated_text)




    ##############################
    # OCR Character Segmentation #
    ##############################
    license_plate_number_easy_ocr = ""
    license_plate_number_pytesseract = ""
    license_plate_number_keras_ocr = ""

    

    for roi in ROI_character_segmentation:
        clean_text=''
        text_tess = ''
        concatenated_text = ''
        # easyOCR
        ocr_results = reader.readtext(roi)

        for (bbox, text, prob) in ocr_results:
            # display the OCR'd text and associated probability
            #print("[INFO] {:.4f}: {}".format(prob, text))

            # cleanup the text and draw the box surrounding the text along
            # with the OCR'd text itself
            clean_text = cleanup_text_easyocr(text)

            license_plate_number_easy_ocr += clean_text

        # pytesseract
        text_tess = pytesseract.image_to_string(roi, lang='eng', config='-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ —psm 8 —oem 3')

        text_tess = ''.join(e for e in text_tess if e.isalnum())  #modify our text no spaces
        license_plate_number_pytesseract += text_tess

        # kerasOCR
        #pipeline = keras_ocr.pipeline.Pipeline()

        image = keras_ocr.tools.read(roi)

        # Realizar predicciones
        try:
            # Perform predictions
            predictions = pipeline.recognize([roi])[0]
        
            texts = [prediction[0] for prediction in predictions]

            # Limpiar los textos
            cleaned_texts = [clean_text_keras(text) for text in texts]

            concatenated_text = ''.join(cleaned_texts)

            license_plate_number_keras_ocr += concatenated_text

        except Exception as e:
            
            license_plate_number_keras_ocr += ''

        

    resultados_easyocr_cs.append(license_plate_number_easy_ocr)
    resultados_pytesseract_cs.append(license_plate_number_pytesseract)
    resultados_kerasocr_cs.append(license_plate_number_keras_ocr)
        
        

In [12]:
for image in imagenes:
    print(image)
    detections = coco_model(image, verbose=True, classes=vehicles)[0]
    detections_ = []

    #detections.save_crop(save_dir='.\\datasets\\validation\\car_detection', file_name=f'image{i}')

    detections = detections.boxes.data.tolist()

    best_detection = None

    best_car_detection = 0.0

    for detection in detections:
        x1, y1, x2, y2, score, class_id = detection

        if score > best_car_detection:
            
            best_detection = detection
            
            best_car_detection = score   

    img = cv2.imread(image)
    img_save = img.copy()


    if best_detection == None:
        image_crop = img_save
    else:
    
        x1, y1, x2, y2, score, class_id = best_detection
        image_crop = img[int(y1):int(y2), int(x1): int(x2)]
    
    lp_predictions = license_plate_detector.predict(source=image_crop, device='cuda', verbose=True)[0]

    #lp_predictions.save(f'.//license_plate_predictions//img{index_file_predict}.jpg')
    #cv2.imshow("Coche detectado", image_crop)

    best_score = 0

    license_plate_found = None

    for detection in lp_predictions.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = detection

        if score > best_score:

            license_plate_ROI = image_crop[int(y1):int(y2),int(x1):int(x2)]

            license_plate_ROI_save = license_plate_ROI

            license_plate_found = detection

            best_score = score
    
    if license_plate_found == None:
        
        lp_predictions = license_plate_detector.predict(source=img_save, device='cuda', verbose=False)[0]
        
        best_score = 0

        license_plate_found = None

        for detection in lp_predictions.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection

            if score > best_score:

                license_plate_ROI = img_save[int(y1):int(y2),int(x1):int(x2)]

                license_plate_ROI_save = license_plate_ROI

                license_plate_found = detection

                best_score = score
    
        if license_plate_found == None:
            resultados_easyocr_og.append('')
            resultados_pytesseract_og.append('')
            resultados_kerasocr_og.append('')

            resultados_easyocr_og_perspective.append('')
            resultados_pytesseract_og_perspective.append('')
            resultados_kerasocr_og_perspective.append('')

            resultados_easyocr_filters.append('')
            resultados_pytesseract_filters.append('')
            resultados_kerasocr_filters.append('')

            resultados_easyocr_cs.append('')
            resultados_pytesseract_cs.append('')
            resultados_kerasocr_cs.append('')

            continue
    
    ## ROI Original
    license_plate_ROI_save = license_plate_ROI.copy()
    
    license_plate_ROI_perspective_corrected = apply_correct_perspective(license_plate_ROI)

    ## ROI Original With Perspective Corrected
    license_plate_ROI_perspective_corrected_save = license_plate_ROI_perspective_corrected.copy()

    ## ROIs Character Segmentation and Filters
    license_plate_ROI_filters, list_of_rois = obtain_license_plate_characters(license_plate_ROI_perspective_corrected)


    ocr_analysis(license_plate_ROI_save, license_plate_ROI_perspective_corrected_save, license_plate_ROI_filters, list_of_rois)


print(resultados_easyocr_og)
print(resultados_pytesseract_og)
print(resultados_kerasocr_og)

print(resultados_easyocr_og_perspective)
print(resultados_pytesseract_og_perspective)
print(resultados_kerasocr_og_perspective)

print(resultados_easyocr_filters)
print(resultados_pytesseract_filters)
print(resultados_kerasocr_filters)

print(resultados_easyocr_cs)
print(resultados_pytesseract_cs)
print(resultados_kerasocr_cs)


../datasets/baza_slika/040603/P1010001.jpg

image 1/1 d:\00. Documentos D\Programacion\TFG\validation\..\datasets\baza_slika\040603\P1010001.jpg: 480x640 1 car, 76.5ms
Speed: 230.4ms preprocess, 76.5ms inference, 1207.5ms postprocess per image at shape (1, 3, 480, 640)

0: 512x640 1 License_Plate, 40.0ms
Speed: 49.0ms preprocess, 40.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)
1/1 [==============================] - 0s 81ms/step
../datasets/baza_slika/040603/P1010002.jpg

image 1/1 d:\00. Documentos D\Programacion\TFG\validation\..\datasets\baza_slika\040603\P1010002.jpg: 480x640 3 cars, 133.0ms
Speed: 14.5ms preprocess, 133.0ms inference, 45.5ms postprocess per image at shape (1, 3, 480, 640)

0: 640x256 (no detections), 43.5ms
Speed: 1.0ms preprocess, 43.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 256)
1/1 [==============================] - 0s 58ms/step
../datasets/baza_slika/040603/P1010003.jpg

image 1/1 d:\00. Documentos D\Programacion\TF

In [13]:
import csv

# List of lists to write to CSV
data_to_write = [
    resultados_easyocr_og,
    resultados_pytesseract_og,
    resultados_kerasocr_og,
    resultados_easyocr_og_perspective,
    resultados_pytesseract_og_perspective,
    resultados_kerasocr_og_perspective,
    resultados_easyocr_filters,
    resultados_pytesseract_filters,
    resultados_kerasocr_filters,
    resultados_easyocr_cs,
    resultados_pytesseract_cs,
    resultados_kerasocr_cs,
]

# Writing data to CSV
with open('ocr_results_baza_slika_rep.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    for row in data_to_write:
        writer.writerow(row)
        